# Probabilistic programming with PyMC

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl

# should be 3.6 or greater
import pymc3 as pm
print("PyMC3 version {version}".format(version=pm.__version__))

import scipy.stats
from pydataset import data

# style matplotlib
mpl.rcParams['figure.figsize'] = (6.0, 2.0)
mpl.rcParams['figure.dpi'] = 120
mpl.style.use('ggplot')

# fix random seed for reproducbility
np.random.seed(2019)

# Neat things

* Demo of MCMC algorithms and their sampling properties: https://chi-feng.github.io/mcmc-demo/
* Hamiltoninan MCMC visually explained (great animations): http://arogozhnikov.github.io/2016/12/19/markov_chain_monte_carlo.html
* [A Conceptual Introduction to Hamiltonian Monte Carlo](https://arxiv.org/abs/1701.02434) An excellent paper on the theory of Hamiltonian Monte Carlo sampling
* [Introduction to MCMC](http://www.inference.org.uk/mackay/erice.pdf) by David Mackay


        
        

# Topic purpose
We will cover probabilistic **inference**. Rather than learning a single set of parameters by optimisation, we can model probability distributions over possible models that might be compatible with our data.  We'll use Monte Carlo sampling to make it simple and easy (if not very efficient) to work with probabilistic models. 


MCMC models:

* **Data**, which we observe as a collection of examples.
* A **model** which has **structure** (a DAG) and **parameters**
* Part of the model is a likelihood function which has "contact" with data; these we will call **observed random variables**
* Part of the model specifies distributions over parameters of the **observed variables**. These are **unobserved random variables**


## PyMC
<a id="pymc"> </a>
We'll use the excellent PyMC module to do the inference. If you have questions about this module, you can read [this tutorial](http://arxiv.org/abs/1507.08050) or the [API docs](https://pymc-devs.github.io/pymc/). 

# Fitting a normal distribution
## Bayesian Normal fitting
We use Monte Carlo sampling to estimate the mean and standard deviation of some data.

We assume we have data generated by a random process where $x \sim \mathcal{N}(\mu, \sigma^2)$, but we don't know $\mu$ or $\sigma$. We can place priors on $\mu$ and $\sigma$ and try and infer a distribution of plausible values.


### Test data
We generate some synthetic data from a known normal distribution. In this case we **know** that our data is in fact normally distributed, so our model assumptions are guaranteed to be correct. This isn't the typical case!!

$$x \sim \mathcal{N}(-1, 1.5)$$


In [ ]:
## generate data with a known distribution
## this will be our "observed" data

n_samples = 30

x_data = np.random.normal(-1,1.5, (n_samples,))

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.hist(x_data, bins=np.linspace(-5,5,15))
ax.set_title("Histogram of data")
ax.set_xlabel("x")
ax.set_ylabel("Count")

We then create a model in PyMC. We have a single output variable `x`, which is **stochastic** and **observed**, and the data we have observed is `x_data`. As it is observed, we will use the likelihood of the data under different model settings to accept/reject samples in the process. 

We use $\tau$ to represent the *reciprocal of variance*, as this is the standard model that PyMC uses. It makes it slightly easier to parameterise in some cases.

We have a model:

$$
\mu \sim \mathcal{N}(0, 10^2)\\
\sigma \sim \text{HalfNormal}(0.0, 10.0)\\
x\sim\mathcal{N}(\mu, \sigma)\\
$$

In [ ]:
import scipy.stats

# plot the PDF of our prior
xs = np.linspace(0, 100, 100)
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

# alpha = 1.0, beta = 20.0
ax.plot(xs, scipy.stats.halfnorm(loc=0, scale=10).pdf(xs))

ax.set_xlabel("$\\sigma$")
ax.set_ylabel("$p(\\sigma)$")
ax.set_title("Std. dev. $\\sigma$ ")


We have to set parents for every node. In this case, we have two parameters, $\mu$ and $\sigma$ to specify. We want to infer those, so we also make those stochastic variables, but unobserved (hidden). We specify the type of the distribution (here, `Normal` and `HalfNormal`) and we must then specify *those* parents. In this case, these are just concrete numbers (but we could go further if we wanted).

In [ ]:
basic_model = pm.Model(name='flexible')

with basic_model:
    # Priors for unknown model parameters
    mu = pm.Normal('mu', mu=0, sd=10)    
    sigma = pm.HalfNormal('sigma', sd=10)

        # Likelihood (sampling distribution) of observations
    y_obs = pm.Normal('y_obs', mu=mu, sd=sigma, observed=x_data)


We can show the graph of this model as a DAG.

In [ ]:
pm.model_to_graphviz(basic_model)

### Sampling
With this model, we can then draw samples. In this case we are going to draw three things:
* Posterior samples (i.e. samples of $\mu$ and $\sigma$)
* Posterior predictive samples (i.e. samples of $y$ that should look like synthetic data)
* Prior predictive samples (i.e. samples of $y$ drawn from the model *without* conditioning on data)
* (we should also draw prior samples, but this isn't currently supported in `pymc3` directly)

In [ ]:

with basic_model:
    trace = pm.sample(500)
    # 50 samples *per* trace element = 50*500 here
    ppc = pm.sample_posterior_predictive(trace, samples=50, model=basic_model)
    # same number of samples from prior
    ppc_prior = pm.sample_prior_predictive(samples=50*500, model=basic_model)


In [ ]:
!theano-cache purge

### Traceplot
The traceplot shows the histogram of posterior samples, and also the trace timeseries. This is useful in debugging poorly mixing chains. There will be no problem fitting in this instance, and everything will look nice.

In [ ]:
pm.traceplot(trace);

### Summary 
We can print a summary of our variables, including:
* mean and sd of posterior
* simulation error
* 95% interval around highest predicted density
* number of effective samples from chain
* Rhat (see below)

In [ ]:
pm.summary(trace)

We can show additional variables, for example the ordinary percentiles of the posterior

In [ ]:
import pandas as pd
def trace_quantiles(x):    
    return pd.DataFrame(pm.quantiles(x, [2.5, 25, 50, 75, 97.5]))
    
pm.summary(trace, stat_funcs = [trace_quantiles])

### Forest plot
We can plot a forest plot, that shows the credible intervals for our parameter estimates,  and also "R-Hat". This is the Gelman-Rubin measure of convergence and it measures how well our sampling process has converged. It is always close to 1.0 and should be very close to 1.0. 

As a rough guide to interpreting `R-hat`
* 1.00 good
* 1.05 worrying
* 1.1 bad things 
* worse than 1.1 sampling has gone badly wrong

In [ ]:
pm.forestplot(trace, varnames=['flexible_mu', 'flexible_sigma']);

# Posterior histograms

In [ ]:
pm.plots.plot_posterior(trace);

In [ ]:
pm.plots.densityplot(trace);

## Autocorrelation
Is our sampler taking uncorrelated samples? We can look at the **autocorrelation** of the samples. If they are perfectly unbiased, then this should be zero everywhere (no correlation between successive samples). We want to draw independent unbiased samples from the posterior, but an MCMC sampling process produces highly correlated samples (each sample depends on the previous). We want to measure and minimise that sample-to-sample correlation, which is captured by the autocorrelation.

We can see how well the HMC chain drew independent samples by looking at the autocorrelation. This should be largely flat.

In [ ]:
pm.plots.autocorrplot(trace);

## Model comparison
Let's compare with fitting a model which has a fixed standard deviation of $\sigma=0.5$.
This is simpler, but obviously a bad model for our data. We can then compare it to our more flexible model.

In [ ]:
fixed_model = pm.Model(name='fixed')

with fixed_model:

    # Priors for unknown model parameters
    mu = pm.Normal('mu', mu=0, sd=10)        

        # Likelihood (sampling distribution) of observations
    y_obs = pm.Normal('y_obs', mu=mu, sd=0.5, observed=x_data)
    fixed_trace = pm.sample(500)

In [ ]:
pm.traceplot(fixed_trace);

In [ ]:
pm.summary(fixed_trace)

In [ ]:
pm.plot_posterior(fixed_trace);

### Information criterions: WAIC and LOO
We can compute the Widely Applicable Information Criterion (WAIC) to get a sense of how well our models compare. This takes into account both the complexity of the models and the goodness-of-fit.  

This is intended to approximate the out-of-sample error, i.e. the generalisation performance. The models with higher "weight" should be considered more viable. In this case, the `flexible` model is vastly better than the fixed model and gets all of the weight. 

In [ ]:
waic_compared = pm.stats.compare({basic_model:trace, fixed_model:fixed_trace})
waic_compared

We can also compute this using the leave-one-out-cross-validation metric. 

In [ ]:
loo_compared = pm.stats.compare({basic_model:trace, fixed_model:fixed_trace}, ic='LOO')
loo_compared

In [ ]:
pm.plots.compareplot(waic_compared)

## Try
Adjusting the fixed $\sigma$ to 1.5 (the true value). The model comparison results will be reversed.

The **trace** is the collection of posterior samples, as a straightforward array. We can plot these using the built in visualisation tool:


We can also access them directly as arrays and plot them more flexibly (including showing draws from the predictive posterior):

In [ ]:
def trace_hist(trace, name):
        fig = plt.figure()
        fig.set_facecolor("white")
        n, bins, patches = plt.hist(trace.ravel(), normed=True, bins=50)                
        max_n = np.max(n)
        plt.title("Estimate of {var_name}".format(var_name=name))
        
        # draw simple statistics
        ctr_max = 0.5 * (bins[np.argmax(n)] + bins[np.argmax(n)+1])
        plt.axvline(ctr_max, ls='-', color='r', lw=2, label='MAP')
        plt.axvline(np.median(trace), ls='-', color='C1', lw=2, label='Median')
        plt.axvline(np.mean(trace), ls=':', color='k', label='Expected')
        # 90% credible interval
        plt.axvline(np.percentile(trace, 5.0), ls=':', color='C1')
        plt.axvline(np.percentile(trace, 95.0), ls=':', color='C1')
        plt.fill_between(x=[np.percentile(trace, 5.0), np.percentile(trace, 95.0)], y1=max_n,
                          color='C1', alpha=0.2, label='90% credible')
        plt.text(np.mean(trace), 0.5*max_n, 'Mean')
        plt.legend()
        plt.gca().set_frame_on(False)
        
def show_trace(mcmc, vars_):
    ## plot histograms of possible parameter values
    # from the trace
    for var,name in vars_.items():
        
        
        trace = mcmc.get_values(var).ravel()
        trace_hist(trace, name)
        
        
def correlate_trace(mcmc, var_a, var_b):
    # plot the correlation between two variables
    # in the posterior as a scatter plot
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.scatter(mcmc.get_values(var_a), 
               mcmc.get_values(var_b), s=1, alpha=0.1)
    ax.set_aspect(1.0)
    ax.set_xlabel(var_a)
    ax.set_ylabel(var_b)
    

In [ ]:

show_trace(trace, {"flexible_mu":"mean", 
                  "flexible_sigma":"std. dev",
                  })

trace_hist(ppc['flexible_y_obs'], "posterior predictive")
trace_hist(ppc_prior['flexible_y_obs'], "prior predictive")

We can see if there are any correlations in the parameters (there probably shouldn't be very strong correlation in this case, though we'd expect the estimated `std_dev` to be higher when the `mean` is further from the true mean).

In [ ]:
correlate_trace(trace, "flexible_mu", "flexible_sigma")

## Changes to try:

* Adjust $n$ to show effect of prior/posterior



# Transformations of variables


Fitting a uniform distribution instead, but using transformed variables. We parameterise in terms of centre and width of a uniform distribution, but transform these variables to the (lower, upper) form that the `Uniform` expects. This is a very simple example of transformations for inference.

$$
c \sim \mathcal{N}(0,10^2)\\
w \sim \text{HalfNormal}(0.0, 5)\\
l = c-w\\
u = c+w\\
x \sim \mathcal{U}(l,u)
$$


In [ ]:
# latent variables
x_data = np.random.uniform(-2, 3, size=80)

np.random.seed(1225)

## WARNING ##
## This model is not well-specified. It is possible for the
# likelihood to be zero for some parameter settings, which breaks
# pymc3 sampling process and is generally a very bad thing to do
# This is why the results are forced into a single
# chain when sampling and the seed is fixed. Inference *does*
# work, kind of.
## WARNING ##
uniform_model = pm.Model()

with uniform_model:
    # Priors for unknown model parameters
    c = pm.Normal('c', mu=0, sd=10)    
    w = pm.HalfNormal('w', sd=5)
    l = c - w
    u = c + w
    y_obs = pm.Uniform('y_obs', lower=l,  upper=u, observed=x_data)

# display the graphical model
pm.model_to_graphviz(uniform_model)    


In [ ]:
with uniform_model:
    trace = pm.sample(2000, cores=1)
     # 50 samples *per* trace element = 50*500 here
    ppc = pm.sample_posterior_predictive(trace, samples=50, model=uniform_model)
    # same number of samples from prior
    ppc_prior = pm.sample_prior_predictive(samples=50*500, model=uniform_model)

In [ ]:
pm.traceplot(trace);

In [ ]:
pm.plots.plot_posterior(trace);

In [ ]:
show_trace(trace, {"c":"centre", 
                  "w":"width"})
trace_hist(ppc['y_obs'], "Posterior predictive")
trace_hist(ppc_prior['y_obs'], "Prior predictive")

---
# Linear regression

## Graphical models
<a id="graphical"> </a>

Transformations of expressions to graphs is familiar to most computer scientists -- it is an essential part of most optimising compilers. For example, the equation of a straight line might be written as a graph (this is how a compiler would break down the expression):

<img src="imgs/ymxc.png" width="300px">

## Adding unknowns
If we have multiple dependent random variables whose distribution we want to infer, we can draw a graph of dependencies to form a *graphical model*.  This explictly models dependencies between **random variables** (i.e. ones we don't know the value of precisely) and inference can be performed on the entire graph. 

**In CS terms, we are writing expressions down without fixing the variables, and then allowing the distribution of the values to be inferred when we observe data.** This inference process narrows down the likely range a random variable could take on (hopefully!).

In a **probabilistic graphical model**, some nodes in the graph are **observed** -- that is we know their state because we have explicity measured it, and others are **unobserved** -- we know (or have guessed) the form of their distribution but not the parameters of that distribution. Some dependencies are deterministic (i.e. fully defined by the values of their parents), while others are stochastic. We can infer the **posterior** distribution of unobserved nodes by integrating over the possible values that could have occured given the observed values.

We can modify our straight line equation to write a model for **linear regression**:

<img src="imgs/ymxc_stochastic.png">

All we need to do is specify that we expected the output $y$ to be normally distributed around the equation of a line given by $m$ and $c$; we can now **infer** $\sigma, m, c$ from observed data. Or we can fix any of them, and infer the remainder (if, e.g. we knew in advance that $c=0$). Our assumption here is that we will observe data which has a **latent structure** modelled by a linear dependence on a variable $x$, plus some normally-distributed observation noise.

**Note that we must put *some* prior distribution on every stochastic node and we can only observe stochastic nodes.**

----


Let's implement the linear regression model in the intro in practice, using PyMC to build a graphical model and then run MCMC to sample from the posterior (i.e. estimate the distribution of random variables after seeing some evidence).

In [ ]:
### Bayesian Linear Regression with pymc
### We use Monte Carlo sampling to estimate the distribution of a linear function with a normally
### distributed error, given some observed data.
### Vaguely based on: http://matpalm.com/blog/2012/12/27/dead_simple_pymc/ and http://sabermetricinsights.blogspot.co.uk/2014/05/bayesian-linear-regression-with-pymc.html


## generate data with a known distribution
## this will be our "observed" data
x = np.sort(np.random.uniform(0,20, (50,)))
m = 2
c = 15

# Add on some measurement noise, with std. dev. 3.0
epsilon = data = np.random.normal(0, 3, x.shape)
y = m * x + c + epsilon

plt.plot(x,y, '.', label="Datapoints")
plt.plot(x, m*x+c, '--', lw=3, label="True")
plt.legend()
plt.xlabel("x")
plt.xlabel("y")

In [ ]:
## Now, set up the PyMC model

regression_model = pm.Model()

with regression_model:
    

    ## specify the prior distribution of the unknown line function variables
    ## Here, we assume a normal distribution over m and c
    m = pm.Normal('m', mu=0, sd=10)
    c = pm.Normal('c', mu=0, sd=50)
    
    std = pm.HalfCauchy('std', beta=20)
    x_obs = pm.Normal('x_obs', mu=0, sd=1, observed=x)
    
    line = m * x_obs + c
    y_obs = pm.Normal('y_obs', mu=line, sd=std, observed=y)
    
    
# display the graphical model
pm.model_to_graphviz(regression_model)

In [ ]:
with regression_model:
    trace = pm.sample(500)
     # 50 samples *per* trace element = 50*500 here
    ppc = pm.sample_posterior_predictive(trace, samples=50, model=uniform_model)
    # same number of samples from prior
    ppc_prior = pm.sample_prior_predictive(samples=50*500, model=uniform_model)

In [ ]:
pm.plots.forestplot(trace)
pm.plots.traceplot(trace)
pm.plots.plot_posterior(trace)
pm.summary(trace)

In [ ]:
trace_hist( ppc["y_obs"], "Posterior predictive")
trace_hist( ppc_prior["y_obs"], "Prior predictive")

## Draws from the posterior predictive model


[<img src="https://imgs.xkcd.com/comics/error_bars.png">](https://xkcd.com/2110)



In [ ]:
## now plot overlaid samples from the linear function
## Note: this *ignores* the error distribution we've estimated
## If we drew samples from the true posterior predictive, 
# we'd see much greater spread
## in possible simulations
ms = trace["m"]
cs = trace["c"]


plt.title("Sampled fits")
plt.plot(x, y, '.', label="Observed")

xf = np.linspace(-20,40,200)
for m,c in zip(ms[::20], cs[::20]):    
    plt.plot(xf, xf*m+c, 'g-', alpha=0.01)
plt.plot(x, x*m+c, '--', label="True", zorder=100)
plt.legend()
plt.xlim(-10,30)
plt.ylim(-20,70)

----


# Logistic regression example: discrete dependent variable
On ye olde iris dataset, using the four flower measurements to predict whether or not
the species is `setosa` or another type of iris.

We estimate a set of coefficients $\beta_0, \beta_1, \dots$ and use the logistic function to transform the a linear model into a probability for a Bernoulli variable.

In [ ]:

from pydataset import data
from sklearn.model_selection import train_test_split

iris = data("iris")
iris["is_setosa"] = np.where(iris["Species"]=="setosa", 1, 0)


In [ ]:
iris.head()

In [ ]:
# split the data into a train and test set
iris_train, iris_test = train_test_split(iris)
print("Train size", iris_train.shape)
print("Test size", iris_test.shape)

## Model:

We have some coefficients $\beta$, which feed into our logistic function to produce $l$, and $y$ is Bernoulli distributed (0 or 1) with probability $l$.

$$
\beta_i \sim \mathcal{N}(0, 5)\\
l = \frac{1}{1+e^{\beta_0 + \sum_i \beta_i x_i}}\\
y \sim \mathcal{B}(l)\\
$$

In [ ]:
# binary prediction of "is_setosa", using the four attributes
# of the flower configuration

# predictors (standardised)
xs = np.array(iris_train.iloc[:, 0:4])
x_standardised = (xs - xs.mean()) / xs.std()

# observed values
ys = np.array(iris_train["is_setosa"])

In [ ]:
logistic_model = pm.Model()

import theano.tensor as tt

with logistic_model:    
    x_std = pm.Normal("x_std", mu=0, sd=1, observed=x_standardised)    
    
    # 4 regression coefficients
    betas = pm.Normal("betas", mu=0, sd=0.1, shape=(5,))
    
    logistic = 1.0 / (1 + tt.exp(-(betas[0] + betas[1] * x_std[:,0] +
                                              betas[2] * x_std[:,1] +
                                              betas[3] * x_std[:,2] +
                                              betas[4] * x_std[:,3]
                                  )))
    
    y = pm.Bernoulli("y", p=logistic, observed=ys)
    
# display the graphical model
pm.model_to_graphviz(logistic_model)

In [ ]:
with logistic_model:
    trace = pm.sample(500, cores=1)

In [ ]:
pm.plots.forestplot(trace)
pm.plots.traceplot(trace)
pm.plots.plot_posterior(trace)
pm.summary(trace)

In [ ]:
fig = plt.figure(figsize=(10,12))

for i in range(5):
    ax = fig.add_subplot(3,2,i+1)    
    trace_hist(trace["betas"][:,i], "$\\beta_{i}$".format(i=i))
    
plt.tight_layout()

In [ ]:
import sklearn.metrics

# write link function for use in prediction
def logistic_predict(betas, x_std):
    return 1.0 / (1 + np.exp(-(betas[0] + np.sum(betas[1:] * x_std, axis=1))))

# standardise predictors in test set
test_x = iris_test.iloc[:, 0:4]
test_x = (test_x - np.mean(test_x))/np.std(test_x)

y_true = iris_test["is_setosa"]

## Predictions
We can draw samples from the posterior and then use the regression coefficients to make new predictions. Annoyingly, 
we have to rewrite the logistic function, but this is easy to do.

In [ ]:
# plot for true versus predicted
fig1 = plt.figure()
ax1 = fig1.add_subplot(1,1,1)
ax1.set_xlabel("True")
ax1.set_ylabel("Predicted")
ax1.set_title("True versus predicted")

# plot for ROC curve
fig2 = plt.figure()
ax2 = fig2.add_subplot(1,1,1)
ax2.set_xlabel("FPR")
ax2.set_xlabel("TPR")

confusions = []
beta_trace = trace["betas"]

# predict 
for i in range(6):
    # choose a random set of betas
    beta_ix = np.random.randint(0, beta_trace.shape[0]-1)
    beta_vec = beta_trace[beta_ix, :]        
    y_pred =  logistic_predict(beta_vec, test_x)    
    ax1.scatter(y_true+np.random.normal(0,0.01,
                                        y_true.shape),
                y_pred,s=20)
    # bias is due to unbalanced classes (I think)
    y_class = np.where(y_pred<0.5, 0, 1)
    confusion = sklearn.metrics.confusion_matrix(y_true, y_class)    
    confusions.append(confusion)
        
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_true, y_pred)
    ax2.plot(fpr, tpr)
        

## Distribution of confusion matrices
We can show the (samples from) distribution of confusion matrices if we want:

In [ ]:
    
fig = plt.figure()
ax1 = fig.add_subplot(1,2,1)
ax1.imshow(np.mean(confusions, axis=0))
ax2 = fig.add_subplot(1,2,2)
ax2.imshow(np.std(confusions, axis=0))

    

In [ ]:
# show samples from the confusion matrices
confusions = np.array(confusions)
# some tensor reshaping fun...
confusion_pad = np.stack([confusions, np.zeros_like(confusions)]).swapaxes(0,1)
flat = np.concatenate(np.concatenate(confusion_pad, axis=0), axis=1)
plt.imshow(flat, cmap='magma')

----

# Switchpoint model: more complex logic

<img src="poverty_rates.png">

*[Source: https://ourworldindata.org/extreme-history-methods]*

Data not provided, so hand-digitised via https://apps.automeris.io/wpd/



In [ ]:
import pandas as pd
from io import StringIO

# load data from a string

data = StringIO("""year,poverty_rate
1819.8097502972653, 83.88791593695271
1849.6789536266351, 81.646234676007
1869.655172413793, 75.48161120840629
1889.821640903686, 71.6987740805604
1909.6076099881093, 65.67425569176883
1928.8228299643283, 56.42732049036777
1949.7502972651605, 54.8861646234676
1959.6432818073722, 44.09807355516638
1969.7265160523186, 35.69176882661996
1979.8097502972653, 31.62872154115587
1991.6052318668253, 23.782837127845866
2004.922711058264, 13.695271453590195
2001.8787158145064, 17.19789842381782
1999.0249702734839, 19.159369527145344
1995.9809750297266, 19.299474605954472
1987.0392390011891, 24.483362521891436
1989.8929845422117, 24.483362521891436
1983.9952437574316, 27.98598949211906
1980.9512485136743, 33.450087565674266
1992.936979785969, 22.521891418563897""")

poverty_ = pd.read_csv(data)
# deleting the dodgy data point
# uncomment to experiment
# poverty = poverty_.drop(labels=[6])

poverty = poverty_
poverty

In [ ]:
poverty.plot(x='year', y='poverty_rate', kind='scatter')
plt.gca().set_frame_on(False)

### Hypothesis
We model the data with a linear regression, but where there is a switchpoint, where the regression coefficient changes (i.e. piecewise linear with two pieces). We estimate both the regression coefficients at each position and the location of the switchpoint.

$$s \sim \mathcal{N}{(1960, 100)}\\
\beta_0 \sim \mathcal{N}(50, 10)\\
\beta_1 \sim \mathcal{N}(-1, 2)\\
\beta_2 \sim \mathcal{N}(-1, 2)\\
$$

$$
\mu = \begin{cases}
x<s & \beta_0 + \beta_1 (x-s)\\
x>s & \beta_0 + \beta_2 (x-s)\\
\end{cases}
$$


$$
\tau \sim \mathcal{\Gamma}(1, 10) \\
y \sim \mathcal{N}(\mu, \frac{1}{\tau})
$$


In [ ]:
switch_model = pm.Model()

import theano.tensor as tt
mean_prate = 50.0 # poverty["poverty_rate"].mean()
mean_year = 1960 # poverty["year"].mean()
with switch_model:    
    year = pm.Normal("year", mu=0, sd=1, observed=poverty["year"]-mean_year)
    # 3 betas
    poverty_at_switchpoint = pm.Normal("poverty_at_switchpoint", mu=0, sd=20.0)
    
    left_slope = pm.Normal("left_slope", mu=0, sd=1)
    right_slope = pm.Normal("right_slope", mu=0, sd=1)
    switchpoint = pm.Normal("switchpoint", mu=0, sd=100)
    
    
    knee = 1.0
    switch_value = (tt.tanh(knee * (switchpoint - year)) + 1) / 2.0
    
    switch_value = (switch_value * (poverty_at_switchpoint+left_slope*(year-switchpoint)) +
    (1-switch_value) * (poverty_at_switchpoint+right_slope*(year-switchpoint)))
                          
    sigma = pm.HalfNormal("sigma", sd=5)
    y = pm.Normal("y", mu=switch_value, sd=sigma, observed=poverty["poverty_rate"]-mean_prate)
    trace = pm.sample(1000)
    
# display the graphical model
pm.model_to_graphviz(switch_model)


In [ ]:
pm.plots.forestplot(trace)
pm.plots.traceplot(trace)
pm.plots.plot_posterior(trace)
pm.summary(trace)
            

In [ ]:
poverty.plot(x='year', y='poverty_rate', kind='scatter')
ax = plt.gca()
ax.set_frame_on(False)
ax.set_ylabel("Global poverty rate")
ax.set_xlim(1800,2020)
ax.set_ylim(0,100)
plt.gcf().set_facecolor('white')

left_slope = trace["left_slope"]
right_slope = trace["right_slope"]
intercept = trace["poverty_at_switchpoint"] + mean_prate
switch_year = trace["switchpoint"] + mean_year


for i in range(1500):
    ix = np.random.randint(0, len(left_slope-1))
    s = switch_year[ix]
    x1 = np.clip(s-200, 1800, 2020)
    x2 = s
    x3 = np.clip(s+200, 1800, 2020)
    y1 = intercept[ix] + left_slope[ix] * (x1-s)
    y2 = intercept[ix] 
    y3 = intercept[ix] + right_slope[ix] * (x3-s)
    ax.plot([x1,x2,x3], [y1,y2,y3], 'C2', lw=0.05, alpha=0.05)
    
    
    
    


----

# A simple mixture model: discrete + continuous latent variables
## When things get tricky

We can include both **discrete** and **continuous** variables. A very important case is where we have a **mixture model**. That is, we believe our observations come from one of a number of distributions. For example, in modelling human heights, we might expect height to be normally distributed, but to have two different distributions for men and women.

<img src="imgs/mixture.png">

It is very straightforward to add this to a PyMC graphical model; it is just another random variable to infer. However, sampling is another matter.

In this case we do full **clustering**.  That is, we suppose the data is generated by three different processes, each of which is normal with some unknown mean and variance, and we have to estimate:

* The parameters of each of $n$ process/clusters
* The index of the cluster/process that each observation belongs to.

This means we have one discrete parameter for *every* data point; we need to label each data point during inference. This is very hard to sample from, as it is a high-dimensional discrete space.

In [ ]:
## Adapted from the example given at 
## http://stackoverflow.com/questions/18987697/how-to-model-a-mixture-of-3-normals-in-pymc

# if you touch this seed, the fit breaks :)
# this is *not* a stable fit with these parameters!
np.random.seed(2028)
n = 3
ndata = 2000


## Generate synthetic mixture-of-normals data, 
# with means at -50,0,+50, and std. dev of 5,10,1
v = np.random.randint( 0, n, ndata)
data = ((v==0)*(np.random.normal(50,5,ndata)) + 
        (v==1)*(np.random.normal(-50,10,ndata)) + 
        (v==2)*np.random.normal(0,1,ndata))


## Plot the original data
plt.hist(data, bins=50);  

In [ ]:
## A Dirichlet distribution specifies the distribution over categories
## All 1 means that every category is equally likely
dd = mc.Dirichlet('dd', theta=(1,)*n)

## This variable "selects" the category (i.e. the normal distribution)
## to use. The Dirichlet distribution sets the prior over the categories.
category = mc.Categorical('category', 
                          p=dd, size=ndata)

## Now we set our priors the precision and mean of each normal distribution
## Note the use of "size" to generate a **vector** of variables 
# (i.e. one for each category)

## We expect the precision of each normal to be Gamma distributed 
# (this mainly forces it to be positive!)
precs = mc.Gamma('precs', alpha=1, 
                 beta=10, size=n)

## And the means of the normal to be normally distributed, with a precision of 0.001 
# (i.e. std. dev 1000)
means = mc.Normal('means', 0, 1.0/(100*100), size=n)

## These deterministic functions link the means of the observed distribution 
# to the categories
## They just select one of the elements of the mean/precision vector, 
# given the current value of category
## The input variables must be specified in the parameters, so that 
# PyMC knows which variables to pass to it
@mc.deterministic
def mean(category=category, means=means):
    return means[category]

@mc.deterministic
def prec(category=category, precs=precs):
    return precs[category]

In [ ]:
## Now we specify the variable we observe -- which is normally distributed, *but*
## we don't know the mean or precision. 
# Instead, we pass the **functions** mean() and pred()
## which will be used at each sampling step.
## We specify the observed values of this node, and tell PyMC these are observed 
## This is all that is needed to specify the model
obs = mc.Normal('obs', mean, prec, 
                value=data, observed = True)

## Now we just bundle all the variables together for PyMC
model = mc.Model({'dd': dd,
              'category': category,
              'precs': precs,
              'means': means,
              'obs': obs})

In [ ]:

show_dag(model)    


In [ ]:
mcmc = mc.MCMC(model)

## Now we tell the sampler what method to use
## Metropolis works well, but we must tell PyMC to use a specific
## discrete sampler for the category variable to get good results in a reasonable time
mcmc.use_step_method(mc.AdaptiveMetropolis, 
                     model.means)
mcmc.use_step_method(mc.AdaptiveMetropolis,
                     model.precs)
mcmc.use_step_method(mc.DiscreteMetropolis, 
                     model.category) ## this step is key!
mcmc.use_step_method(mc.AdaptiveMetropolis, 
                     model.dd)

## Run the sampler with 5 different chains
mcmc.sample(iter=150000, burn=1000)


In [ ]:
plt.figure()
plt.hist(mcmc.trace('means', chain=None).gettrace()[:], normed=True, bins=np.linspace(-100,100,50))
plt.title("Estimated means")
plt.legend(['Component 1', 'Component 2', 'Component 3'])
plt.figure()
## show the result in terms of std. dev. (i.e sqrt(1.0/precision))
plt.title("Estimated std. dev")
plt.hist(np.sqrt(1.0/mcmc.trace('precs', chain=None).gettrace()[:]), normed=True, 
         bins=np.linspace(0,15,50))
plt.legend(['Component 1', 'Component 2', 'Component 3'])

# Mixture modelling without classification

If all we wanted to do was to estimate the parameters of the mixture (i.e. the PDF), and *not* perform the clustering process that assigns labels to datapoints, then we can write a simpler model ("marginalising out the discrete variables"). We write a custom stochastic variable representing a mixture-of-Gaussian likelihood function with vector parameters. This then lets us estimate the distribution but does not identify the classes each data point belongs to. This has no discrete parameters and is easier to fit. We can try and do this class labeling post hoc, assigning each observation to the most probable class, but this loses the uncertainty about class membership that we have in the full model above.



In [ ]:
import theano.tensor as tt
with pm.Model() as model:
    w = pm.Dirichlet('w', np.ones(n))
    
    mu = pm.Normal('mu', 0., 10., shape=n, testval=[-10, 0, 10])
    sigma = pm.HalfCauchy('sd', 20, shape=n)
    
    # trick: force the order of mu to be sequential, so that the posterior
    # samples are stable across multiple chains. This makes the likelihood 0
    # if the order of mu is not increasing
    p = pm.Potential('order', tt.switch(tt.extra_ops.diff(mu)<0, -np.inf, 0).sum())
    
    x_obs = pm.NormalMixture('x_obs', w, mu, sd=sigma, observed=data)
    trace = pm.sample(1000, cores=1)

In [ ]:
pm.plots.traceplot(trace);
pm.plots.plot_posterior(trace);
pm.stats.summary(trace)

In [ ]:
import theano.tensor as tt

    
with pm.Model() as model:
    w = pm.Dirichlet('w', np.ones(n))    
    mu_0 = pm.Normal('mu_0', 0., 50.)
    mu_n = pm.HalfNormal('mu_n',  50., shape=n-1)
    sigma = pm.HalfCauchy('sd', 10, shape=n)    
    # alternative; force mus to be increasing by nature
    mu_transform = pm.Deterministic('mu_s', tt.cumsum(tt.concatenate([tt.stack(mu_0), mu_n])))
    x_obs = pm.NormalMixture('x_obs', w, mu_transform, sd=sigma, observed=data)
    trace = pm.sample(1000, cores=1)

In [ ]:
pm.plots.traceplot(trace);
pm.plots.plot_posterior(trace);
pm.stats.summary(trace)

In [ ]:
bins = np.linspace(-100,100,200)

plt.hist(mcmc.trace("means")[:,0], bins=bins);
plt.hist(mcmc.trace("means")[:,1], bins=bins);
plt.hist(mcmc.trace("means")[:,2], bins=bins);



In [ ]:
bins = np.linspace(0,20,50)
to_std = lambda x: np.sqrt(1.0/x)
plt.hist(to_std(mcmc.trace("precs")[:,0]))
plt.hist(to_std(mcmc.trace("precs")[:,1]))
plt.hist(to_std(mcmc.trace("precs")[:,2]))


# Imputation in quadratic regression
<a id="imputation"> </a>

In PyMC, variables can be **observed** (fixed) or **unobserved** (random). PyMC cycles through the array of known values for the **observed** variables and updates the rest of the graph.


PyMC implements this using **imputation**, where certain missing values in an observed variable can be inferred (*imputed*) from the rest of the model. This creates new random variables and then infers the missing values. **Masked arrays** are used to implement imputation; these allow arrays to have "blank" values, that PyMC can fill in automatically.

This approach creates one new random variable per missing data item; this can create very large models if you are not careful!




In [ ]:
## Example, using very simple quadratic regression model 
import numpy.ma as ma # masked array support

## generate the data for the regression
x = np.sort(np.random.uniform(0, 20, (50,)))
m = 2
c = 15
# Add on some measurement noise, with std. dev. 3.0
epsilon = data = np.random.normal(0, 50, x.shape)
y = m * x * x + c + epsilon

In [ ]:
## Now the imputation; we will try and infer missing some missing values of y (we still have the corresponding x)
## mark last three values of y invalid
y_impute = y[:]

n_missing = 6
impute_ixs = np.sort(np.random.randint(0, len(y)-1, size=n_missing))
y_impute[impute_ixs] = 0
y_impute = ma.masked_equal(y_impute,0)
print("Y masked for imputation:", y_impute) # we will see the last three entries with --

In [ ]:
# create the model (exactly as before, except we switch "y_impute" for "y")

with pm.Model() as model:
    m_inf = pm.Normal('m', mu=0, sd=10)    
    c_inf = pm.Normal('c', mu=0, sd=50)    
    std = pm.HalfNormal('std', sd=50)
    x_obs = pm.Normal('x_obs', observed=x)
    y_obs = pm.Normal('y_obs', mu=m_inf*x_obs*x_obs+c_inf, sd=std, observed=y_impute)
    trace = pm.sample(1000, cores=1)
    

In [ ]:
pm.plots.traceplot(trace);
pm.plots.plot_posterior(trace);
pm.stats.summary(trace)

In [ ]:
## now we will have three entries in the y_obs trace from this run

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

## the original data
ax.plot(x, y, '.', label="Data")

ax.plot(x, x*x*m+c, ':', label="True")

m_sample = trace["m"]
c_sample = trace["c"]

# plot samples from the quadratic fits

for i in range(50):
    m_post = np.random.choice(m_sample)
    c_post = np.random.choice(c_sample)
    ax.plot(x, x*x*m_post + c_post, "g", alpha=0.05,
           label="Posterior" if i==0 else None)
    
imputed = trace.get_values('y_obs_missing')

# samples from posterior predicted for the missing values of y
for i in range(len(impute_ixs)):
        
    ax.axvline(x[impute_ixs[i]], c='C1', alpha=0.1, label="Imputed" if i==0 else None)
    # plot the actual imputed data points
    ax.scatter(np.tile(x[impute_ixs[i]], 
                        (len(imputed), 1)), 
                        imputed[:,i], s=2, c='C3', marker='_', 
                        alpha=0.25)

ax.set_xlim(-1,25)
ax.set_xticks(np.arange(0,25,5))
ax.set_xticklabels(np.arange(0,25,5))
ax.legend()    

------------